In [1]:
import sys
import numpy as np
from tqdm import tqdm_notebook
import pickle
import json as js
import pandas as pd
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import time
import init as x
import multiprocessing as mp
from functools import partial
import helpers as hp

## setting
alpha_const = True
alpha_val = 1 #0.7
R0 = x.R0_default

beta_mixed = True
run_inout = False

## setup
out_filename_dir,out_fig_dir,out_stat_dir=hp.setup_paths(R0,alpha_val,"")
out_filename = out_filename_dir+"/"+x.out_filename
out_filename_info = out_filename_dir+"/info.json"

# choose distribution for beta and gammma
if run_inout:
    import simulate_corona_inout as sc
else:
    import simulate_corona_seir as sc
    
# alpha-strategy
if alpha_const:
    alpha_mat = np.full([x.N_locs,x.N_per],alpha_val)
else:
    idx_sel = np.arange(0,x.N_locs)
    init_sel = .7
    tar_sel = 1
    per = 50
    init_rem = 1
    tar_rem = 1
    alpha_mat = hp.create_alpha_matrix(idx_sel,init_sel,tar_sel,per,init_rem,tar_rem)

# get random vectors and rescale R0 if needed
R0_scale = R0/x.R0_default
beta_list,Tinf_inv_list,Tinc_inv_list = x.get_vectors(R0_scale,True)

## initialize
rep_vec = np.arange(0,x.N_simul)

def run_simulation(beta_list,Tinf_inv_list,Tinc_inv_list,alpha_mat):
    pfun = partial(sc.simul,beta_list,Tinf_inv_list,Tinc_inv_list,alpha_mat)
    cores_cnt = mp.cpu_count()
    run_cnt = int(np.ceil(x.N_simul/cores_cnt))
    res = []
    dt_avg = 0
    for run_idx in np.arange(run_cnt):
        pool = mp.Pool(cores_cnt)
        tic = time.perf_counter()
        res0 = pool.map(pfun,np.arange(0,cores_cnt))
        # res = sc.simul(beta_list,Tinf_inv_list,Tinc_inv_list,alpha_mat,0)
        toc = time.perf_counter()
        pool.close()
        dt_avg += (toc-tic)
        res = res+res0
    dt_avg /= x.N_simul
    return res,dt_avg 

def save_results(filename,data,stat=False):
    if stat:       
        pct_S_mean = hp.mean_list(data,'S')
        pct_E_mean = hp.mean_list(data,'E')
        pct_C_mean = hp.mean_list(data,'C')
        pct_R_mean = hp.mean_list(data,'R')
        pct_S_q95 = hp.quant_list(data,'S',.95)
        pct_E_q95 = hp.quant_list(data,'E',.95)
        pct_C_q95 = hp.quant_list(data,'C',.95)
        pct_R_q95 = hp.quant_list(data,'R',.95)        
        pct_S_q05 = hp.quant_list(data,'S',.05)
        pct_E_q05 = hp.quant_list(data,'E',.05)
        pct_C_q05 = hp.quant_list(data,'C',.05)
        pct_R_q05 = hp.quant_list(data,'R',.05)
        data_stat = pd.DataFrame(list(zip(pct_S_mean, pct_E_mean, pct_C_mean,pct_R_mean, pct_S_q95,pct_E_q95,pct_C_q95,pct_R_q95,pct_S_q05,pct_E_q05,pct_C_q05,pct_R_q05)), 
                         columns = ['S_mean','E_mean','C_mean','R_mean','S_q95','E_q95','C_q95','R_q95','S_q05','E_q05','C_q05','R_05'])
        with open(filename+"_stat.pickle","wb") as fid_stat:
            pickle.dump(data_stat,fid_stat)
        fid_stat.close()
    with open(filename+".pickle",'wb') as fid:
        pickle.dump(data,fid)
    fid.close()
    dic = {"run_seir":True,"R0":R0,"alpha":alpha_val,"inout":False,"beta":beta_mixed}
    with open(out_filename_info,"w") as fid:
        js.dump(dic,fid)
    fid.close()

if alpha_const:
    print("Alpha: "+str(np.round(alpha_val,3)))
else:
    print("Alpha: plan")
print("R0:"+str(np.round(R0,3))+"\t (default value = "+str(np.round(x.R0_default,3))+")")
print("Beta tail follows Power distribution: "+str(beta_mixed))

print("Running simulation ... ",end="",flush=True)
#data,tpi=run_simulation(beta_list,Tinf_inv_list,Tinc_inv_list,alpha_mat)
sc.dsim()
print("Finished")
#print("Time per iteration (sec.): "+str(tpi))

## save results
print("Saving results ...",end="",flush=True)

save_results(out_filename,data,stat=True)
print("Finished")    
    

Alpha: 1
R0:1.359	 (default value = 1.359)
Beta tail follows Power distribution: True
Running simulation ... [6.5, 1.9000000000000004, 1.1053283334143473]
[0.9996708091505437, 4.02311893232258e-05, 0.0002889596601330046]
[0.9996667454963276, 2.3120533369369344e-05, 0.00031013397030312277]
[0.9996635543167136, 1.4143011210178358e-05, 0.00032230267207647526]
[0.9996629309426056, 7.322695207559266e-06, 0.00032974636218709543]
[0.9996625787759584, 3.820811745704137e-06, 0.0003336004122963374]
[0.9996624001851679, 1.988448985615053e-06, 0.000335611365846708]
[0.9996623294415239, 1.0126405319879515e-06, 0.0003366579179443998]
[0.9996622942085791, 5.149047758484477e-07, 0.0003371908866454461]
[0.9996622682815588, 2.6982928254061615e-07, 0.00033746188915905086]
[0.9996622682815588, 1.27813870677134e-07, 0.000337603904570914]
[0.9996622682815588, 6.054341242601085e-08, 0.0003376711750291656]
[0.9996622682815588, 2.8678458517584115e-08, 0.00033770303998307383]
[0.9996622682815588, 1.358453298201

[0.9996622682815588, 1.2185597504464632e-43, 0.0003377317184415912]
[0.9996622682815588, 5.772125133693776e-44, 0.0003377317184415912]
[0.9996622682815588, 2.7341645370128425e-44, 0.0003377317184415912]
[0.9996622682815588, 1.295130570163978e-44, 0.0003377317184415912]
[0.9996622682815588, 6.134829016566209e-45, 0.0003377317184415912]
[0.9996622682815588, 2.905971639426104e-45, 0.0003377317184415912]
[0.9996622682815588, 1.3765128818334172e-45, 0.0003377317184415912]
[0.9996622682815588, 6.520324177105661e-46, 0.0003377317184415912]
[0.9996622682815588, 3.0885746102079456e-46, 0.0003377317184415912]
[0.9996622682815588, 1.463009025887975e-46, 0.0003377317184415912]
[0.9996622682815588, 6.930042754206196e-47, 0.0003377317184415912]
[0.9996622682815588, 3.282651830939777e-47, 0.0003377317184415912]
[0.9996622682815588, 1.5549403409714747e-47, 0.0003377317184415912]
[0.9996622682815588, 7.365506878285932e-48, 0.0003377317184415912]
[0.9996622682815588, 3.4889243107670193e-48, 0.0003377317

NameError: name 'data' is not defined

In [ ]:
# Basic statistics
# 1. Peak of infectious, 95% CI
scaling_factor = 1/x.first_infections_correction_multiplier
C_mean = hp.mean_list(data,'C',ma=True,w=6)*scaling_factor
R_mean = hp.mean_list(data,'R',ma=True,w=6)*scaling_factor
S_mean = hp.mean_list(data,'S',ma=True,w=6)*scaling_factor
print(S_mean)
C_q95 = hp.quant_list(data,'C',0.95,ma=True,w=10)*scaling_factor
C_q05 = hp.quant_list(data,'C',0.05,ma=True,w=10)*scaling_factor
C_med = np.array(hp.quant_list(data,'C',val=0.50,ma=True,w=10))*scaling_factor
C_mean_abs = np.round(x.N_popul_size*C_mean)
C_q95_abs = np.round(x.N_popul_size*C_q95)
C_q05_abs = np.round(x.N_popul_size*C_q05)
C_med_abs = np.round(x.N_popul_size*C_med)
print("Basic Statistics: number of observed infectious people, R0 = "+ str(np.round(R0,2)), 'Day 0 = 24.3.2020')
print('Peak, mean value: ',np.round(C_mean_abs.max()),'(',np.round(100*C_mean.max(),3),'% of pop.)', ' day: ',C_mean_abs.argmax())
print('Peak, median value: ',np.round(C_med_abs.max()),'(',np.round(100*C_med.max(),3),'% of pop.)',' day: ',C_med_abs.argmax())
print('Peak, confidence interval at 95% : ',np.round(C_q95_abs.max()),'(',np.round(100*C_q95.max(),3),'% of pop.)',' day: ',C_q95_abs.argmax())
print('Peak, confidence interval at 5% : ',np.round(C_q05_abs.max()),'(',np.round(100*C_q05.max(),3),'% of pop.)',' day: ',C_q05_abs.argmax())    

xval=np.arange(0,x.N_per)
plt.plot(xval,C_mean_abs,color='blue',linewidth=2,alpha=.9)
plt.plot(xval,C_q95_abs,color='blue',linestyle='--',alpha=.8)
plt.plot(xval,C_q05_abs,color='blue',linestyle='-.',alpha=.8)
plt.fill_between(xval,C_q05_abs,C_q95_abs,color='blue',alpha=0.25)
plt.legend(['mean', '95%', '5%'])
plt.xlabel("days")
plt.title("Observed Infectious (95% CI), R0 = "+ str(np.round(R0,2)))
plt.savefig(out_fig_dir+"/peaks.png",dpi=300)

In [ ]:
# 2. report changes in number of infected
def smooth(arr,w=8):
    arr =  pd.DataFrame(arr)
    arr = arr.apply(np.mean,1)
    xx = arr.rolling(window=w).mean()
    xx[0:w-1] = arr.rolling(window=2).mean()[0:w-1]
    arr[1:]=xx[1:]    
    return arr

def get_stat(data,colname,pop,fact):
    x_mean = pd.array(hp.mean_list(data,colname,ma=True,w=10))
    x_q95 = pd.array(hp.quant_list(data,colname,0.95,ma=True,w=10))
    x_q05 = pd.array(hp.quant_list(data,colname,0.05,ma=True,w=10))
    x_med = pd.array(np.array(hp.quant_list(data,colname,val=0.50,ma=True,w=10)))
    x_mean_abs = ((pop*x_mean)*fact)
    x_q95_abs = (pop*x_q95)*fact
    x_q05_abs = (pop*x_q05)*fact
    x_med_abs = (pop*x_med)*fact
    dx_mean_abs = smooth(x_mean_abs[1:x.N_per]-x_mean_abs[0:x.N_per-1])
    dx_q95_abs  = smooth(x_q95_abs[1:x.N_per]-x_q95_abs[0:x.N_per-1])
    dx_q05_abs  = smooth(x_q05_abs[1:x.N_per]-x_q05_abs[0:x.N_per-1])
    dx_mean_pct = smooth(100*(x_med_abs[1:x.N_per]/x_med_abs[0:x.N_per-1]-1))
    dx_q95_pct  = smooth(100*(x_q95_abs[1:x.N_per]/x_q95_abs[0:x.N_per-1]-1))
    dx_q05_pct  = smooth(100*(x_q05_abs[1:x.N_per]/x_q05_abs[0:x.N_per-1]-1))
    x_mean_abs = smooth(x_mean_abs)
    return x_mean_abs, x_med, x_q95_abs,x_q05_abs,dx_mean_abs,dx_q95_abs,dx_q05_abs

        
C_mean_abs,C_med,C_q95_abs,C_q05_abs,dC_mean_abs,dC_q95_abs,dC_q05_abs=get_stat(data,'C',x.N_popul_size,scaling_factor)
E_mean_abs,E_med,E_q95_abs,E_q05_abs,dE_mean_abs,dE_q95_abs,dE_q05_abs=get_stat(data,'E',x.N_popul_size,scaling_factor)
S_mean_abs,S_med,S_q95_abs,S_q05_abs,dS_mean_abs,dS_q95_abs,dS_q05_abs=get_stat(data,'S',x.N_popul_size,scaling_factor)
R_mean_abs,R_med,R_q95_abs,R_q05_abs,dR_mean_abs,dR_q95_abs,dR_q05_abs=get_stat(data,'R',x.N_popul_size,scaling_factor)
C_in = dE_mean_abs-0*dS_mean_abs#np.round(+x.N_popul_size*(np.array(hp.mean_diff_list(data,'E'))+np.array(hp.mean_diff_list(data,'S'))))
C_out = dR_mean_abs #np.round(x.N_popul_size*np.array(hp.mean_diff_list(data,'R')))

xval = np.arange(1,x.N_per)
plt.subplot(121)
plt.plot(xval,dC_mean_abs,color='blue',linewidth=2,alpha=.9)
plt.plot(xval,dC_q05_abs,color='blue',linestyle='--',alpha=.8)
plt.plot(xval,dC_q95_abs,color='blue',linestyle='-.',alpha=.8)
# plt.plot(xval,C_in,color='red',linewidth=2,alpha=.9)
plt.fill_between(xval,dC_q05_abs,dC_q95_abs,color='blue',alpha=0.25)
plt.title('Change in number of contagenous (daily basis, absolute numbers)')

plt.subplot(122)
xval = np.arange(0,x.N_per)
#plt.plot(xval,C_mean_abs,color='green',linewidth=2,alpha=.9)
#plt.plot(xval,E_mean_abs,color='yellow',linewidth=2,alpha=.9)
#plt.plot(xval,S_mean_abs,color='red',linewidth=2,alpha=.9)
plt.plot(xval,R_mean_abs,color='orange',linewidth=2,alpha=.9)
# plt.plot(xval,C_out,color='red',linestyle='--',alpha=.9)
plt.legend(['inflow','outflow'])

plt.savefig(out_fig_dir+"/change.png",dpi=300)
